In [ ]:
import math
import sympy
import time

# Cache prime blocks
known_primes = list([2])

# Get the last known prime
last_known_prime = lambda: known_primes[len(known_primes) - 1]


# The below algorithm isn't a primality checker, it calculate the next prime
# after the last_known_prime in the known_primes list (i.e. for a prime p, gives p + 1).
#
# Unlike a traditional approach, this is not optimised for speed.
# It is much faster to combine seives & statistical primality checks
# before falling back on a slower deterministic primality algorithm.
#
# This algorithm determines the next prime directly, by looking at
# modular remainders of trial divisors in a set of known primes, i.e:
#
#   -last_known_prime % [set of known primes < sqrt of last_known_prime]
#
# These remainders show the position in each known_prime's modular revolution
# at the last_known_prime. As we are using a negative dividend, the remainders
# equal the next number where each trial divisor's remainder will reach 0,
# meaning the integer at that location can not be prime.
#
# After cycling the whole set of primes we know all the positions that are
# blocked, and inversely the next position which is free (where no remainders are 0)
# which is the next prime.
#
# In other words instead of progressively checking if each subsequent integer is a prime,
# we determine which remainders don't exist in a set of trial divisors < sqrt of known prime.
# A missing remainder shows that there is an unknown factor that isn't in our set of
# trial divisors. We know all factors of a number have to be <= the square root of the number,
# so this missing remainder is the prime gap which tells us exactly where the next prime is.
#
# There is probably a fun way to more rapidly find the next position
# where a set of numbers is not equal to a specific value without so much
# iteration. I'll keep working on it.
#
# Some ideas
# - Signal processing / fourier transforms:
#       I've found adding stacks of sin(remainder / n) reveals primes
#       Could be an entry point to analyse the algorithm for real numbers
# - Is there a non-iterative algorithm that, given a set of cycling numbers,
#       can determine when *none* of those equal 0?
def calculate_next_known_prime():
    # How it works: We know every integer > 1 is a potential "slot" where a prime number
    # might exist. The algorithm finds modular remainders of known primes up to
    # sqrt(last_known_prime). By cycling these remainders forward from last_known_prime
    # we progressively block the slots where a prime *might* exist. After all possible slots
    # have been blocked, the first empty slot is the next prime number.
    # Given any prime number, it will determine the gap to the next one without foreknowledge of
    # the next prime.
    blocked_prime_slots = set()
    max_range = math.ceil(math.sqrt(last_known_prime()))

    # Repeat cycles to find all blocked prime gaps up to max last_known_prime
    for prime_number in known_primes:
        # We don't need to check primes greater than the sqrt of the last known prime
        if prime_number > max_range:
            break
        # Cycle each prime number's modular remainders to cover all potential slots in max_range
        cycle = 0
        while max_range >= (blocked_prime_gap := (-last_known_prime() % prime_number) + (prime_number * cycle)):
            # blocked_prime_gap indicates a position where the remainder = 0, so an integer
            # at that position can't be prime. We repeat the cycle for smaller divisors so all gaps are found.
            # We are not looking ahead of the current prime directly, only at the last known prime and
            # previous known primes. We do not cycle ahead of sqrt(last_known_prime). We find p+1 only by
            # relation to p.
            blocked_prime_slots.add(blocked_prime_gap)
            cycle += 1

    # Check to find the first missing slot.
    # As we are starting from a known prime (besides 2) the gap is always even
    # We *can* adjust the algorithm to work with the gap of 1 between 2 and 3
    # But that's the only odd gap so it's faster to add it to the known_primes manually
    check_empty_slot = 2
    while blocked_prime_slots.__contains__(check_empty_slot):
        check_empty_slot += 2

    # Redundant variable, just to clarify that we have found the next prime gap
    # So prime (p + 1) is last_known_prime + prime_gap
    prime_gap = check_empty_slot

    # The empty slot must be prime, so add it to the found prime list
    known_primes.append(last_known_prime() + prime_gap)
    return last_known_prime()


# This is a generalised version of the above, which can find the next prime number
# greater than n.
# It follows a similar approach, but instead of only cycling the set of primes through
# one epicycle
#


def next_prime_any(n):

    # Return nearest highest prime to n if it's a known prime
    for p in known_primes:
        if n < p:
            return p

    # We only need up to sqrt(n) primes to find the prime after n
    max_range = math.ceil(math.sqrt(n))

    # Repeat finding primes until max_range (which is sqrt(n))
    # We need all primes up to sqrt(n) to find the next empty slot after n
    while last_known_prime() <= max_range:
        calculate_next_known_prime()

    # Find the next prime after n
    blocked_prime_slots = set()
    for prime_number in known_primes:
        cycle = 0
        while math.ceil(n / prime_number) >= (blocked_prime_gap := (-n % prime_number) + (prime_number * cycle)):
            blocked_prime_slots.add(blocked_prime_gap)
            cycle += 1

    check_empty_slot = 1 if n % 2 == 0 else 2
    while blocked_prime_slots.__contains__(check_empty_slot):
        check_empty_slot += 2
    # Return the next prime guess
    return n + check_empty_slot


start = time.time()
our_guess = next_prime_any(test_n)
our_time = time.time() - start
print(f"our guess: {our_guess} (time: {our_time:.6f} seconds)")
start = time.time()
sympy_guess = sympy.nextprime(test_n)
sympy_time = time.time() - start
print(f"sympy guess: {sympy_guess} (time: {sympy_time:.6f} seconds)")

print(known_primes)

our guess: 10000019 (time: 0.669339 seconds)
sympy guess: 10000019 (time: 0.000000 seconds)
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 125, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1

In [93]:
a = 10
normal_remainders = []
reversed_remainders = []
reversed_equivalent = []
for b in range(0,a):
    normal_remainders.append(b % a)
    reversed_remainders.append(-b % a)
    reversed_equivalent.append((a - b % a) if b % a > 0 else 0)
print("Normal remainders", normal_remainders)
print("Reversed remainders", reversed_remainders)
print("Reversed equivalent", reversed_equivalent)

Normal remainders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Reversed remainders [0, 9, 8, 7, 6, 5, 4, 3, 2, 1]
Reversed equivalent [0, 9, 8, 7, 6, 5, 4, 3, 2, 1]
